# T matrix for electromagnetic scattering

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
h = 0.05 # Size of the mesh
d = [1,0,0] # wave's travel direction
k = 5 # wavenumber

In [3]:
def spherical_harm(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)    

In [12]:
def spherical_harm_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = spherical_harm(p, q, x)
    y3 = spherical_harm(-p, q, x)
    
    if abs(p) == q and p >= 0:
        return (p / np.tan(polar)) * y1 
    elif abs(p) != q and p >= 0:
        y2 = spherical_harm(p + 1, q, x)
        return (p / np.tan(polar)) * y1 + np.sqrt((q - p) * (q + p + 1)) * np.exp(-1j * azimuth) * y2
    elif abs(p) == q and p < 0:
        return (-p / np.tan(polar)) * y3 * np.exp(1j*p*azimuth*2) / (-1)**(-p)
    elif abs(p) != q and p < 0:
        y4 = spherical_harm(-p + 1, q, x)
        return ((-p / np.tan(polar)) * y3 + np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(-1j * azimuth) * y4) * np.exp(1j*p*azimuth*2) / (-1)**(-p)

def spherical_harm_dazimuth(p, q, x):
    return 1j * p * spherical_harm(p, q, x)

def spherical_bessel_dr(q,x):
    r = np.linalg.norm(x)
    return k*(spherical_bessel(q-1,x) - ((q+1)/(k*r))*spherical_bessel(q,x))

In [17]:
def div_free_vector_spherical(p,q,x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    r_ = (spherical_bessel(q,x)/np.sin(polar)) * (0.5 * np.sin(2*polar)*spherical_harm_dpolar(p,q,x) + np.cos(2*polar)*spherical_harm(p,q,x)) - spherical_bessel(q,x)* (np.cos(azimuth)*spherical_harm(p,q,x) + np.sin(azimuth)*spherical_harm_dazimuth(p,q,x))
    polar_ = spherical_bessel(q,x) * (-np.sin(azimuth)*spherical_harm(p,q,x) + np.cos(azimuth)*spherical_harm_dazimuth(p,q,x)) - np.cos(polar)*(2*spherical_bessel(q,x) + r*spherical_bessel_dr(q,x))
    azimuth_ = np.sin(polar)*np.sin(azimuth)*spherical_harm(p,q,x)*(2*spherical_bessel(q,x)+r*spherical_bessel_dr(q,x)) - np.cos(azimuth)*spherical_bessel(q,x)*(np.cos(polar)*spherical_harm(p,q,x) + np.sin(polar)*spherical_harm_dpolar(p,q,x))
    return [r_*np.sin(polar_)*np.cos(azimuth_), r_*np.sin(polar_)*np.sin(azimuth_), r_*np.cos(polar)]